<a href="https://colab.research.google.com/github/sekihiro/Colabo/blob/master/bert_pre_trained_models.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 【PyTorch】BERTの使い方 - 日本語pre-trained models
- 事前学習した日本語pre-trained modelsの精度を確認します。
- 今回はMasked Language Modelの精度を確認します。
- Masked Language Modelを簡単に説明すると、文の中のある単語をマスクしておき、そのマスクされた単語を予測するというものです。
- https://qiita.com/kenta1984/items/7f3a5d859a15b20657f3

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## defaultでインストールされていないライブラリを入れる
- MeCab と transformers(旧名：pytorch-pretrained-BERT)

In [0]:
# MeCab
!apt install aptitude
!aptitude install mecab libmecab-dev mecab-ipadic-utf8 git make curl xz-utils file -y
!pip install mecab-python3

In [0]:
# transformers(旧名：pytorch-pretrained-BERT)
!pip install transformers

In [5]:
import os
import sys
import pprint
import torch
from transformers import BertJapaneseTokenizer, BertForMaskedLM, BertForSequenceClassification

## tokenizer と model の取得
- 東北大学 乾・鈴木研究室の作成・公開されたBERTモデル
- 公開されているモデルは４種類あるが、bert-base-japanese-whole-word-masking を使うのが良い
- 日本語Wikipediaを用いて学習
- okenizerはMeCab + WordPiece（character tokenizationもある）
- max sequence lengthは512
- https://qiita.com/nekoumei/items/7b911c61324f16c43e7e

### transformers で定義されているクラス
- http://kento1109.hatenablog.com/entry/2019/08/20/161936
- BertForMaskedLM : 単語を出力するためのクラス
- BertForSequenceClassification : 分類問題のためのクラス

In [6]:
BASE_PATH = "/content/drive/My Drive/git/"
MODEL_PATH = BASE_PATH + 'model/bert/base-japanese-whole-word-masking/'

if os.path.isfile(MODEL_PATH + 'vocab.txt'):
    print('loading bert-pytorch_tokenizer ..... ')
    tokenizer = BertJapaneseTokenizer.from_pretrained(MODEL_PATH)
else:
    print('downloading bert-pytorch_tokenizer ..... ')
    tokenizer = BertJapaneseTokenizer.from_pretrained('bert-base-japanese-whole-word-masking')
    tokenizer.save_pretrained(MODEL_PATH)

if os.path.isfile(MODEL_PATH + 'pytorch_model.bin'):
    print('loading bert-pytorch_model ..... ')
    model = BertForMaskedLM.from_pretrained(MODEL_PATH)
else:
    print('downloading bert-pytorch_model ..... ')
    model = BertForMaskedLM.from_pretrained('bert-base-japanese-whole-word-masking')
    model.save_pretrained(MODEL_PATH)

loading bert-pytorch_tokenizer ..... 
loading bert-pytorch_model ..... 


In [7]:
# Tokenize input
text = 'テレビでサッカーの試合を見る。'
#text = '今日は仕事で疲れたので、飲んで帰った。'
print(text)
tokenized_text = tokenizer.tokenize(text)
# ['テレビ', 'で', 'サッカー', 'の', '試合', 'を', '見る', '。']

# Mask a token that we will try to predict back with `BertForMaskedLM`
masked_index = 2
tmp = ''
for i, buf in enumerate(tokenized_text):
    if i == masked_index:
        tmp = tmp + '[MASK]'
    else:
        tmp = tmp + tokenized_text[i]
print(tmp)
tokenized_text[masked_index] = '[MASK]'
# ['テレビ', 'で', '[MASK]', 'の', '試合', 'を', '見る', '。']

# Convert token to vocabulary indices
indexed_tokens = tokenizer.convert_tokens_to_ids(tokenized_text)
# [571, 12, 4, 5, 608, 11, 2867, 8]

# Convert inputs to PyTorch tensors
tokens_tensor = torch.tensor([indexed_tokens])
# tensor([[ 571,   12,    4,    5,  608,   11, 2867,    8]])

# Predict
model.eval()
with torch.no_grad(): # 必要のない計算を停止 パラメータの保存を止める
    outputs = model(tokens_tensor)
    ##### outputs ===> [1, 8, 32000]
    #pprint.pprint(outputs)
    #pprint.pprint(outputs[0])
    #pprint.pprint(outputs[0][0])
    #pprint.pprint(outputs[0][0][masked_index]) # [MASK]の部分の予測値を得る
    predictions = outputs[0][0][masked_index].topk(10) # 予測結果の上位N件を抽出(値とインデックスを得る)
    #print('----------')
    #pprint.pprint(predictions)

# Show results
print('----------\n[MASK]')
for i, index_t in enumerate(predictions.indices): # インデックスのみ採用
    index = index_t.item() # tensor -> int
    token = tokenizer.convert_ids_to_tokens([index])[0]
    print(i, token)

テレビでサッカーの試合を見る。
テレビで[MASK]の試合を見る。
----------
[MASK]
0 クリケット
1 タイガース
2 サッカー
3 メッツ
4 カブス
5 の
6 ライオンズ
7 レッズ
8 ヤンキース
9 ジャイアンツ
